<a href="https://colab.research.google.com/github/menouarazib/llm/blob/main/Evaluate_Phi_multi_steps_reasoning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q datasets tqdm

# Install peft, transformers, and trl from Github
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q git+https://github.com/huggingface/peft
!pip install -q git+https://github.com/huggingface/trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 3.1 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

# Load dataset from the hub
dataset = load_dataset("gsm8k", 'main')
eval_dataset = dataset['test']

print(eval_dataset)

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer'],
    num_rows: 1319
})


In [3]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

# Specify the model ID
peft_model_id = "Menouar/phi-2-basic-maths"

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

adapter_config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


adapter_model.safetensors:   0%|          | 0.00/755M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM',

In [ ]:
from tqdm import tqdm

result_sep = '####'
not_clear_answers = 0

def get_final_result(gen_answer):
    if result_sep in gen_answer:
        l = gen_answer.split(result_sep)
        for item in l[1:]:  # Skip the first item, as it is before the first '####'
            item = item.strip()  # Remove leading and trailing whitespace
            if item != '' and item != '\n':
                return item
    return None

def add_prompt(sample, use_prompt):
    if use_prompt:
        return f"{sample['question']}\n\nGiven that the model is trained on grade school math problems, let's approach this step by step:\n"
    else:
        return sample['question']


def evaluate(sample):
    prompt = add_prompt(sample, False)
    answer = sample["answer"]
    outputs = pipe(prompt, max_new_tokens=256, do_sample=False, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    predicted_answer = outputs[0]['generated_text']
    true_value = get_final_result(answer)
    predicted_value = get_final_result(predicted_answer)
    print(f"True value: {true_value} => Predicted value: {predicted_value}")

    if predicted_value is None:
      global not_clear_answers  # Access global variable here
      not_clear_answers += 1
      return 0
    elif true_value in predicted_value:
        return 1
    else:
      return 0

success_rate = []
number_of_eval_samples = len(eval_dataset)

# iterate over eval dataset and predict
for s in tqdm(eval_dataset.select(range(number_of_eval_samples))):
    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)
not_clear_answers_ratio = not_clear_answers/len(success_rate)

print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Unclear answers: {not_clear_answers_ratio*100:.2f}%")

  0%|          | 1/1319 [00:18<6:46:35, 18.51s/it]

True value: 18 => Predicted value: 20


  0%|          | 2/1319 [00:33<6:06:47, 16.71s/it]

True value: 3 => Predicted value: 6 bolts


  0%|          | 3/1319 [00:49<5:59:15, 16.38s/it]

...

True value: 122 => Predicted value: 122 crabs


 14%|█▎        | 180/1319 [46:49<4:49:50, 15.27s/it]

True value: 130 => Predicted value: 130$


 14%|█▎        | 181/1319 [47:04<4:48:46, 15.23s/it]

True value: 20 => Predicted value: 20 votes
